In [ ]:
import pandas as pd

In [ ]:
df_2024 = pd.read_csv("/content/combined_data_2024.csv")
#df_2023 = pd.read_csv("/content/combined_data.csv")

In [ ]:
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554119 entries, 0 to 554118
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   YEAR             554119 non-null  int64  
 1   MONTH            554119 non-null  int64  
 2   DAY              554119 non-null  int64  
 3   HOUR             554119 non-null  int64  
 4   MINUTE           554119 non-null  int64  
 5   SHIFT            554119 non-null  object 
 6   METHOD           554119 non-null  object 
 7   OFFENSE          554119 non-null  object 
 8   BLOCK            554119 non-null  object 
 9   WARD             554119 non-null  int64  
 10  ANC              554119 non-null  object 
 11  DISTRICT         554119 non-null  int64  
 12  PSA              554119 non-null  int64  
 13  VOTING_PRECINCT  554119 non-null  object 
 14  LATITUDE         554119 non-null  float64
 15  LONGITUDE        554119 non-null  float64
dtypes: float64(2), int64(8), object(6)
mem

In [ ]:
#df_2023.info()

In [ ]:
df_2024["OFFENSE"].unique()

array(['THEFT', 'ASSAULT W/DANGEROUS WEAPON', 'ROBBERY',
       'MOTOR VEHICLE THEFT', 'BURGLARY', 'ARSON', 'HOMICIDE',
       'SEX ABUSE'], dtype=object)

In [ ]:
df_2024["OFFENSE"] = df_2024["OFFENSE"].replace({"ROBBERY": "THEFT", "BURGLARY": "THEFT"})

In [ ]:
df_2024["OFFENSE"].unique()

array(['THEFT', 'ASSAULT W/DANGEROUS WEAPON', 'MOTOR VEHICLE THEFT',
       'ARSON', 'HOMICIDE', 'SEX ABUSE'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Define X (features) and y (target)
x_independent = df_2024.drop(columns=['OFFENSE', 'BLOCK'])  # Drop the target variable and DAY column
y_dependent = df_2024['OFFENSE']  # Target variable

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_dependent)

# Define categorical and numerical columns
categorical_cols = ['SHIFT', 'METHOD', 'ANC', 'VOTING_PRECINCT']  # Categorical features
numerical_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'WARD', 'DISTRICT', 'PSA', 'LATITUDE', 'LONGITUDE']  # Numerical features

# One-Hot Encode Categorical Variables using pd.get_dummies()
# drop_first=True avoids dummy variable trap
X_encoded = pd.get_dummies(x_independent, columns=categorical_cols, drop_first=True)

# Combine the encoded categorical columns with numerical columns (already included in X_encoded)

# Split the data: 90% training, 10% testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.10, random_state=42)

# Print shapes of the training and test sets
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")


X_train shape: (498707, 203), X_test shape: (55412, 203)
y_train shape: (498707,), y_test shape: (55412,)


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
from xgboost import XGBClassifier

# Create and configure the XGBoost model
model_xgb = XGBClassifier(
    use_label_encoder=False,         # Disable the label encoder warning
    eval_metric='mlogloss',         # Set evaluation metric for multi-class classification
    random_state=42,                # For reproducibility
    n_estimators=500,               # Increase the number of boosting rounds
    learning_rate=0.05,             # Lower learning rate
    max_depth=6,                    # Adjust max depth (experiment with values like 3, 5, or 7)
    min_child_weight=1,             # Minimum child weight to prevent overfitting
    subsample=0.8,                  # Use 80% of the training data for each tree
    colsample_bytree=0.8,           # Use 80% of features for each tree
    scale_pos_weight=None,            # Set this if you have imbalanced classes
    tree_method='gpu_hist'
)
# Fit the model
model_xgb.fit(X_train, y_train, verbose=True)

# Make predictions
y_pred = model_xgb.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:48:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:48:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:49:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:49:02] WARNING: /workspace/src/common/

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.70      0.43      0.53      3276
           2       0.93      0.94      0.93       252
           3       0.44      0.00      0.01      5945
           4       0.00      0.00      0.00       387
           5       0.85      0.99      0.91     45514

    accuracy                           0.84     55412
   macro avg       0.49      0.39      0.40     55412
weighted avg       0.79      0.84      0.78     55412



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=1000, random_state=42)
# Fit the model
model_lr.fit(X_train, y_train)

# Make predictions
y_pred = model_lr.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.63      0.19      0.30      3276
           2       0.00      0.00      0.00       252
           3       0.00      0.00      0.00      5945
           4       0.00      0.00      0.00       387
           5       0.83      0.99      0.91     45514

    accuracy                           0.83     55412
   macro avg       0.24      0.20      0.20     55412
weighted avg       0.72      0.83      0.76     55412



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

In [ ]:
import lightgbm as lgb

model_lgb = lgb.LGBMClassifier()
# Fit the model
model_lgb.fit(X_train, y_train)

# Make predictions
y_pred = model_lgb.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1118
[LightGBM] [Info] Number of data points in the train set: 498707, number of used features: 203
[LightGBM] [Info] Start training from score -7.290828
[LightGBM] [Info] Start training from score -2.792949
[LightGBM] [Info] Start training from score -5.349973
[LightGBM] [Info] Start training from score -2.239277
[LightGBM] [Info] Start training from score -4.989126
[LightGBM] [Info] Start training from score -0.198472
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.69      0.41      0.52      3276
           2       0.81      0.88      0.84       252
           3       0.20 

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

model_cbc = CatBoostClassifier(silent=True, random_state=42)
# Fit the model
model_cbc.fit(X_train, y_train)

# Make predictions
y_pred = model_cbc.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.71      0.43      0.53      3276
           2       0.93      0.93      0.93       252
           3       0.52      0.01      0.02      5945
           4       0.00      0.00      0.00       387
           5       0.85      0.99      0.91     45514

    accuracy                           0.84     55412
   macro avg       0.50      0.39      0.40     55412
weighted avg       0.80      0.84      0.79     55412



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier()
# Fit the model
model_knn.fit(X_train, y_train)

# Make predictions
y_pred = model_knn.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.20      0.06      0.10      3276
           2       0.60      0.69      0.64       252
           3       0.20      0.07      0.10      5945
           4       0.00      0.00      0.00       387
           5       0.83      0.95      0.89     45514

    accuracy                           0.80     55412
   macro avg       0.30      0.30      0.29     55412
weighted avg       0.72      0.80      0.75     55412



In [ ]:
from sklearn.naive_bayes import GaussianNB

model_naivebayes = GaussianNB()
# Fit the model
model_naivebayes.fit(X_train, y_train)

# Make predictions
y_pred = model_naivebayes.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.84      0.00        38
           1       0.31      0.43      0.36      3276
           2       0.57      0.75      0.65       252
           3       0.17      0.11      0.13      5945
           4       0.01      0.05      0.02       387
           5       0.91      0.26      0.40     45514

    accuracy                           0.25     55412
   macro avg       0.33      0.41      0.26     55412
weighted avg       0.79      0.25      0.37     55412

